In [0]:
CREATE OR REFRESH STREAMING TABLE bronze_orders
COMMENT 'raw orders ingested data'
TBLPROPERTIES ('quality'='bronze')
AS SELECT *, 
          _metadata.file_path as input_file_path,
          current_timestamp as ingestion_timestamp
FROM cloud_files(
  '/Volumes/circuitbox/landing/operational_data/orders/',
  'json',
  map('cloudFiles.inferColumnTypes','true')
)

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_orders_clean(
  CONSTRAINT valid_customer_id EXPECT (customer_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_order_id EXPECT (order_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_order_status EXPECT (order_status IN ('Pending','Shipped','Cancelled','Completed')),
  CONSTRAINT valid_payment_method EXPECT (payment_method IN ('Credit Card','Bank Transfer','Paypal'))
)
COMMENT 'cleaned customers data'
TBLPROPERTIES ('quality'='silver')
AS
SELECT order_id, 
       customer_id,
       CAST(order_timestamp AS timestamp) AS order_timestamp,
       payment_method,
       items,
       order_status
FROM STREAM(LIVE.bronze_orders)

In [0]:
CREATE STREAMING TABLE silver_orders
AS
SELECT order_id, 
       customer_id,
       order_timestamp,
       payment_method,
       order_status,
       item.item_id,
       item.name AS item_name,
       item.category AS item_category,
       item.price AS item_price,
       item.quantity AS item_quantity
FROM (SELECT order_id, 
       customer_id,
       order_timestamp,
       payment_method,
       order_status,
       explode(items) AS item
  FROM STREAM(LIVE.silver_orders_clean));